cell to import packages: numpy, matplotlib, tensorflow, keras, sklearn.metrics

In [1]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  

from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
from sklearn import metrics

Using TensorFlow backend.


In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "ankle_acc_x_",
    "ankle_acc_y_",
    "ankle_acc_z_",
    "upperleg_acc_x_",
    "upperleg_acc_y_",
    "upperleg_acc_z_",
    "trunk_acc_x_",
    "trunk_acc_y_",
    "trunk_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "NON-FREEZING",
    "FREEZING"
] 

## Preparing Dataset

In [3]:
# file directory, 
#train data and test set are all in the directory
# 2/3 data for training, 1/3 data for testing
#TRAIN = '../../dataset_fog_release/dataset/Train/'
TRAIN = 'D:/FoG/dataset_fog_release/dataset/Train/'

# parameters
# prediction and RNN parameter, related to how to arrange the data
predict_d = 300  # how many samples in the forward FOG prediction
Tx = 100 # number of time steps in the RNN
Tstep = 3 # RNN examines data every Tstep entries

Tcutoff = 6000 # Examine at most 1500*0.015 seconds before FoG
Sstep = 6 # sample step is 6

n_a=128 # RNN internal size

## New code to load data Data Loading

In [4]:
# the program is modified from Prof. Chakarborty's program
#
# create the X and Y values from the files 
#
#.  1. remove entries not in the experiments 
#.  2. Experimental data are partitioned into episodes
#.  3. Generate batch of Tx entries for each episode (X) 
#.     with marking (Y), and concatenate all data for 
#.     training and testing.

e_count = 0 # count the number of episodes

def shuffle_list(a, b):
    assert len(a)==len(b)
    s_a = []
    s_b = []
    for i in np.random.permutation(len(a)):
        s_a += [a[i]]
        s_b += [b[i]]
    return s_a, s_b
    

# generate X, Y for one episode of Fog or end of file
def gen_batch_one_episode(data, beg, end):
    X = []
    Y = []
    XX = []
    YY = []
    i = end - 1;
    while data[i][9] == '2':
        i -= 1
    
    # cut off if necessary
    if i-Tcutoff > beg:
        beg = i-Tcutoff
    
    for i in [n for n in range(beg, end-Tx*Tstep, Sstep)]:
        t = []
        for j in range(Tx):
            t += [data[i+j*Tstep][:9]]
        X += [t]
        flag = 0;
        for j in range(Tx):
            if i+j*Tstep + predict_d > end-1: 
                break
            if data[i+j*Tstep+predict_d][9] == '2':
                flag = 1
                break
        if flag == 1: 
            Y +=[[0,1]]
        else : 
            Y += [[1, 0]]
    
    X1, Y1 = shuffle_list(X, Y)
    
    #for i in range(len(X)):
    #    if Y[i] == [0, 1]:
    #        XX += [X[i]]
    #        YY += [Y[i]]
    return X1, Y1

# generate X, Y for all episode
def gen_batch(data):
    global e_count
    X = []
    Y = []
    X1 = []
    Y1 = []
    beg = 0
    end = beg
    print('len=', len(data))
    while beg < len(data):
        end = beg
        while end < len(data) and data[end][9] != '2':
            # print(end, data[end], '**', data[end][9])
            end = end + 1
        while end < len(data) and data[end][9] == '2':
            end = end + 1
        if data[end-1][9] == '2': 
            e_count = e_count+1
            
        X1, Y1 = gen_batch_one_episode(data, beg, end)
        X += X1
        Y += Y1
        beg = end
    return X, Y
        
def load_data(file_paths):
    global e_count
    train = []
    test = []
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []
    batch_size = 0
    for root, dirs, filenames in os.walk(file_paths):
        for fffDir in filenames:
            X_signals = []
            file = open(file_paths + fffDir, 'r')
            for row in file:
                batch_signals = []
#                print(row.strip().split(' ')[1:])
                if int(row.strip().split(' ')[1:][9]) != 0:
                    batch_signals += [row.strip().split(' ')[1:]]
                    X_signals += batch_signals
            file.close()
            
            train += X_signals[:2*int(len(X_signals)/3)]
            test += X_signals[2*int(len(X_signals)/3):]
            #train += X_signals[len(X_signals)-120:len(X_signals)-60]
            #test += X_signals[len(X_signals)-60:]
    
    train = np.array(train)
    train_X, train_Y = gen_batch(train)
    print(e_count, ' episodes in training data')
    e_count = 0
    test_X, test_Y = gen_batch(test)
    print(e_count, ' episodes in testing data')
    return train_X, train_Y, test_X, test_Y

train_X, train_Y, test_X, test_Y = load_data(TRAIN)
train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
#train_Y = np.reshape(train_Y, (len(train_Y), 1))
test_Y = np.array(test_Y)
#test_Y = np.reshape(test_Y, (len(test_Y), 1))
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

len= 822416
165  episodes in training data
len= 411221
90  episodes in testing data
(54788, 100, 9)
(54788, 2)
(30880, 100, 9)
(30880, 2)


In [5]:
sample = open('output.txt', 'w')
np.set_printoptions(threshold=sys.maxsize)
print(train_Y, file=sample)
sample.close()

# Build the RNN
   Code is copied and modified from the coursera course

In [6]:
n_values = 9
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state=True)
densor = Dense(2, activation='softmax')

In [7]:
def mymodel(Tx, n_a, n_values): 
    """
    Implement the model
    Arguments: 
    Tx -- length of the time steps
    n_a -- the number of activations in the model
    n_values -- number of values as the input per time step
    
    Returns: 
    model -- a keras instance model with n_a activations
    """
    
    #define the input layer and specify the shape
    X = Input(shape=(Tx, n_values))
    
    #define the initial hidden state a0
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    # Step 1: Create empty list to append the outputs while you iterate
    outputs = []
    
    # Step 2: Loop
    
    for t in range(Tx):
        
        x = Lambda(lambda z: z[:, t, :])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(inputs = x, initial_state = [a, c])
        if t==Tx-1:
             out=densor(a)
             outputs.append(out)
    #print(outputs)
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    return model

In [8]:
model = mymodel(Tx=Tx, n_a=n_a, n_values = 9)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 9)       0                                            
__________________________________________________________________________________________________
lambda_100 (Lambda)             (None, 9)            0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_99 (Lambda)              (None, 9)            0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_98 (Lambda)              (None, 9)            0           input_1[0][0]                    
____________________________________________________________________________________________

In [9]:
opt = Adam(lr=0.01, beta_1 = 0.9, beta_2=0.999, decay = 0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# initialize hidden state and cell state

In [10]:
XXX=train_X
YYY = train_Y

a0 = np.zeros((len(XXX), n_a))
c0= np.zeros((len(XXX), n_a))
print(YYY.shape)

(54788, 2)


In [11]:
model.fit([XXX, a0, c0], YYY, batch_size=32, epochs=20)

Epoch 1/20
54788/54788 [==============================] - 650s 12ms/step - loss: 0.4813 - accuracy: 0.7607
Epoch 2/20
54788/54788 [==============================] - 584s 11ms/step - loss: 0.3955 - accuracy: 0.8230
Epoch 3/20
54788/54788 [==============================] - 556s 10ms/step - loss: 0.3683 - accuracy: 0.8395
Epoch 4/20
54788/54788 [==============================] - 557s 10ms/step - loss: 0.3507 - accuracy: 0.8491
Epoch 5/20
54788/54788 [==============================] - 558s 10ms/step - loss: 0.3410 - accuracy: 0.8537
Epoch 6/20
54788/54788 [==============================] - 556s 10ms/step - loss: 0.3348 - accuracy: 0.8571
Epoch 7/20
54788/54788 [==============================] - 556s 10ms/step - loss: 0.3308 - accuracy: 0.8585
Epoch 8/20
54788/54788 [==============================] - 561s 10ms/step - loss: 0.3280 - accuracy: 0.8595
Epoch 9/20
54788/54788 [==============================] - 557s 10ms/step - loss: 0.3251 - accuracy: 0.8618
Epoch 10/20
54788/54788 [============

In [12]:
#model.save('rnn_model.h5')

#model1 = load_model('rnn_model.h5')
Out = model.predict([XXX, a0, c0], verbose=0)

## try the model on the train and test data

In [13]:
Out1 = np.around(Out)
print(Out1.shape[0])
print(Out1.shape)

c0=0
c1=0
c2=0
for i in range(Out1.shape[0]):
    if YYY[i][0] == 0:
        c0 += 1
    if Out1[i][0] != YYY[i][0]:
        if Out1[i][0] == 0:
            c1 += 1
        else : 
            c2 += 1
            
print('c0 = ', c0)
print('c1 = ', c1)
print('c2 = ', c2)

54788
(54788, 2)
c0 =  17349
c1 =  2725
c2 =  4496


In [14]:
NXXX=test_X
NYYY=test_Y

a1 = np.zeros((len(NXXX), n_a))
c1= np.zeros((len(NYYY), n_a))
print(NYYY.shape)
NOut = model.predict([NXXX, a1, c1], verbose=0)

NOut1 = np.around(NOut)
print(NOut1.shape[0])
print(NOut1.shape)

Nc0=0
Nc1=0
Nc2=0
for i in range(NOut1.shape[0]):
    if NYYY[i][0] == 0:
        Nc0 += 1
    if NOut1[i][0] != NYYY[i][0]:
        if NOut1[i][0] == 0:
            Nc1 += 1
        else : 
            Nc2 += 1
            
print('Nc0 = ', Nc0)
print('Nc1 = ', Nc1)
print('Nc2 = ', Nc2)

(30880, 2)
30880
(30880, 2)
Nc0 =  9434
Nc1 =  3424
Nc2 =  3369


In [15]:
model.save('rnn_model_100_2_128_20.h5')